**PENCARIAN DAN PENAMBANGAN WEB - TUGAS 2 : Membuat VSM dan Menghitung TF-IDF berdasarkan hasil crawling berita**

Nama : Isnita Widyur Rahmah
NIM : 220411100048
Kelas : IF 7A

Link Project : https://github.com/taawdyy/ppw

**Crawling Berita Online (CNN Indonesia)**

Crawling berita adalah proses otomatisasi pengambilan data dari situs web berita atau sumber informasi lainnya untuk tujuan analisis, penyimpanan, atau penyajian informasi. Proses ini dilakukan oleh program atau skrip yang dikenal sebagai web crawler atau spider

Import Library

In [97]:
pip install requests beautifulsoup4 pandas


In [87]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

requests digunakan untuk mengambil halaman web, BeautifulSoup untuk mengekstrak informasi dari konten HTML, dan pandas untuk menyimpan dan mengolah data yang diambil.

Mendefinisikan URL

In [98]:
category_urls = [
    'https://www.cnnindonesia.com/ekonomi/indeks/5',
    'https://www.cnnindonesia.com/olahraga/indeks/7'
]

Mengumpulkan Data

In [106]:
def crawl_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Mengambil judul
    title_element = soup.find('h1', class_='text-cnn_black')
    title = title_element.get_text().strip() if title_element else 'Judul tidak ditemukan'

    # Mengambil Isi
    content_div = soup.find('div', class_='detail-text')
    if content_div:
        paragraphs = content_div.find_all('p')
        content = "\n".join([p.get_text().strip() for p in paragraphs])
    else:
        content = 'Isi artikel tidak ditemukan'

    # Mengambil tanggal
    date_div = soup.find('div', class_='text-cnn_grey text-sm mb-4')
    date_text = date_div.text.strip() if date_div else 'Tanggal tidak ditemukan'

    # Mengambil kategori
    cattegory_div = soup.find('a', class_="text-sm text-cnn_black_light3 hover:text-cnn_red gtm_breadcrumb_kanal")
    cattegory = cattegory_div.text.strip() if cattegory_div else 'Kategori tidak ditemukan'

    return {'Judul': title, 'Isi': content, 'Tanggal': date_text, 'Kategori': cattegory}

Melakukan Crawling Web

In [107]:
def crawl_page(url, article_count, max_articles, articles):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    article_links = soup.find_all('a', class_='flex group items-center gap-4')
    for link in article_links:
        if article_count >= max_articles:
            return article_count
        article_url = link['href']
        article = crawl_article(article_url)
        articles.append(article)
        article_count += 1

    next_button = soup.find('a', class_='inline-flex items-center justify-center w-[30px] h-[30px]')
    next_url = next_button['href'] if next_button else None

    if next_url and article_count < max_articles:
        return crawl_page(next_url, article_count, max_articles, articles)

    return article_count


Mengumpulkan Artikel Berita dari berbagai Kategori

In [108]:
def crawl_category(url, max_articles):
    articles = []
    article_count = 0
    print(f'\nMengambil data dari {url}')
    article_count = crawl_page(url, article_count, max_articles, articles)
    return articles

Mengambil Artikel Berita dari beberapa Kategori yang telah ditentukan dalam category_urls.

In [109]:
max_articles_per_category = 50
all_articles = []

for url in category_urls:
    articles = crawl_category(url, max_articles_per_category)
    all_articles.extend(articles)


Mengambil data dari https://www.cnnindonesia.com/ekonomi/indeks/5

Mengambil data dari https://www.cnnindonesia.com/olahraga/indeks/7


Konversi ke dalam DataFrame

In [110]:
df = pd.DataFrame(all_articles)
df

,Judul,Isi,Tanggal,Kategori
0,Pertemuan Sri Mulyani-Prabowo Tak Banyak Bahas...,Wakil Menteri Keuangan II Thomas Djiwandono me...,"Rabu, 11 Sep 2024 18:10 WIB",Ekonomi
1,Pebisnis Minta Jokowi Cabut Larangan Jual Roko...,Gabungan pengusaha rokok dan petani tembakau m...,"Rabu, 11 Sep 2024 17:31 WIB",Ekonomi
2,IHSG Melemah Tipis ke 7.760 Sore Ini,Indeks Harga Saham Gabungan (IHSG) ditutup di ...,"Rabu, 11 Sep 2024 16:37 WIB",Ekonomi
3,Rupiah Menguat Rp15.402 per Dolar AS Usai Deba...,Nilai tukar rupiah berada di level Rp15.402 pe...,"Rabu, 11 Sep 2024 16:24 WIB",Ekonomi
4,Sri Mulyani Usai Nonton Timnas-Australia: Teri...,Menteri Keuangan Sri MulyaniÂ berkomentar soal...,"Rabu, 11 Sep 2024 15:47 WIB",Ekonomi
...,...,...,...,...
95,Hasil Liga 1: PSM vs Persib Sama Kuat,PSM Makassar harus puas berbagi satu angka usa...,"Rabu, 11 Sep 2024 17:25 WIB",Olahraga
96,"Jokowi Beri Bonus Rp36,25 Miliar ke Peraih Med...",Presiden Joko Widodo (Jokowi) menyerahkan bonu...,"Rabu, 11 Sep 2024 17:13 WIB",Olahraga
97,Megawati Ungkap Target di Liga Korea: Jadi Pem...,Megawati Hangestri Pertiwi mengungkapkan targe...,"Rabu, 11 Sep 2024 16:49 WIB",Olahraga
98,Media Vietnam: Indonesia Buat Kejutan Besar La...,Media Vietnam memuji performa Timnas Indonesia...,"Rabu, 11 Sep 2024 16:24 WIB",Olahraga


Simpan dalam bentuk CSV

In [111]:
df.to_csv('crawl_berita.csv', index=False, encoding='utf-8')